In [31]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType
import datetime as dt
from pyspark.sql import functions as F
from pyspark.sql import Window

import os

# Config

In [30]:
# ==========================================
# 사용자 설정 (원하는 지역과 시군구를 입력하세요)
# ==========================================
REGION = "경기도"          # "ex.서울특별시"
SIGUNGU = "용인시 처인구"    # "ex.강남구"

# 베이스 경로
OUTPUT_BASE = "/opt/spark/data/output/silver_stage_1"
ADDRESS_BASE = "/opt/spark/data/address/parquet"
BUILDING_BASE   = "/opt/spark/data/buildingLeader/parquet"
TOJI_BASE       = "/opt/spark/data/tojiSoyuJeongbo/parquet"
REST_OWNER_BASE = "/opt/spark/data/restaurant_owner/parquet"


# 데이터 필터용(축약형) 매핑
REGION_MAP = {
    "서울특별시": "서울",
    "부산광역시": "부산",
    "대구광역시": "대구",
    "인천광역시": "인천",
    "광주광역시": "광주",
    "대전광역시": "대전",
    "울산광역시": "울산",
    "세종특별자치시": "세종",
    "경기도": "경기",
    "강원도": "강원",
    "충청북도": "충북",
    "충청남도": "충남",
    "전라북도": "전북",
    "전라남도": "전남",
    "경상북도": "경북",
    "경상남도": "경남",
    "제주특별자치도": "제주",
}

REGION_SHORT = REGION_MAP[REGION]
print("선택된 REGION 축약형: ", REGION_SHORT)

# 데이터 필터용(축약형) 매핑
REGION_ENG_MAP = {
    "서울특별시": "seoul",
    "부산광역시": "busan",
    "대구광역시": "daegu",
    "인천광역시": "incheon",
    "광주광역시": "gwangju",
    "대전광역시": "daejeon",
    "울산광역시": "ulsan",
    "세종특별자치시": "sejong",
    "경기도": "gyunggi",
    "강원도": "gangwon",
    "충청북도": "chungbuk",
    "충청남도": "chungnam",
    "전라북도": "jeonbuk",
    "전라남도": "jeonnam",
    "경상북도": "gyeongbuk",
    "경상남도": "gyeongnam",
    "제주특별자치도": "jeju",
}

REGION_ENG = REGION_ENG_MAP[REGION]
print("선택된 REGION 영어: ", REGION_ENG)

선택된 REGION 축약형:  경기
선택된 REGION 영어:  gyunggi


In [3]:
# Spark Session 설정
spark = SparkSession.builder \
    .appName(f'silver_stage_1_{REGION}_{SIGUNGU.replace(" ", "_")}') \
    .master("spark://spark-master:7077") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "400") \
    .getOrCreate()

Picked up JAVA_TOOL_OPTIONS: -Dfile.encoding=UTF-8
Picked up JAVA_TOOL_OPTIONS: -Dfile.encoding=UTF-8
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/16 10:14:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 원천 데이터 로드 + 지역/시군구 필터링

In [4]:
address_path = f"{ADDRESS_BASE}/rnaddrkor_{REGION_ENG}.parquet"
address_df = spark.read.option("mergeSchema", "false").parquet(address_path)

address_df.printSchema()

address_filtered_df = (
    address_df
    .filter(F.col("시도명") == REGION)
    .filter(F.col("시군구명") == SIGUNGU)
    .withColumn(
        "pnu_land_gb",
        F.when(F.col("산여부") == 1, F.lit("2")).otherwise(F.lit("1"))
    )
    .withColumn(
        "PNU코드",
        F.concat(
            F.col("법정동코드"),
            F.col("pnu_land_gb"),
            F.lpad(F.col("지번본번(번지)").cast("string"), 4, "0"),
            F.lpad(F.coalesce(F.col("지번부번(호)"), F.lit(0)).cast("string"), 4, "0")
        )
    )
    .select(
        F.col("PNU코드").cast("string"),

        F.concat_ws(
            " ",
            F.col("시도명"),
            F.col("시군구명"),
            F.col("도로명"),
            F.concat(
                F.col("건물본번").cast("string"),
                F.when(
                    (F.col("건물부번").isNotNull()) & (F.col("건물부번") != 0),
                    F.concat(F.lit("-"), F.col("건물부번").cast("string"))
                ).otherwise(F.lit(""))
            )
        ).alias("도로명주소")
    )
)

SIGUNGU_CODE = address_filtered_df.select(F.substring("PNU코드", 1, 5).alias("sigungu")).first()["sigungu"]

address_filtered_df.printSchema()
print(address_filtered_df.count())
address_filtered_df.show(5, truncate=False)
print("SIGUNGU_CODE =", SIGUNGU_CODE)

root
 |-- 도로명주소관리번호: string (nullable = true)
 |-- 법정동코드: long (nullable = true)
 |-- 시도명: string (nullable = true)
 |-- 시군구명: string (nullable = true)
 |-- 법정읍면동명: string (nullable = true)
 |-- 법정리명: string (nullable = true)
 |-- 산여부: long (nullable = true)
 |-- 지번본번(번지): long (nullable = true)
 |-- 지번부번(호): long (nullable = true)
 |-- 도로명코드: long (nullable = true)
 |-- 도로명: string (nullable = true)
 |-- 지하여부: long (nullable = true)
 |-- 건물본번: long (nullable = true)
 |-- 건물부번: long (nullable = true)
 |-- 행정동코드: double (nullable = true)
 |-- 행정동명: string (nullable = true)
 |-- 기초구역번호(우편번호): long (nullable = true)
 |-- 이전도로명주소: double (nullable = true)
 |-- 효력발생일: double (nullable = true)
 |-- 공동주택구분: long (nullable = true)
 |-- 이동사유코드: double (nullable = true)
 |-- 건축물대장건물명: string (nullable = true)
 |-- 시군구용건물명: string (nullable = true)
 |-- 비고: double (nullable = true)



root
 |-- PNU코드: string (nullable = true)
 |-- 도로명주소: string (nullable = false)

41534
+-------------------+------------------------------------+
|PNU코드            |도로명주소                          |
+-------------------+------------------------------------+
|4146110100100060009|경기도 용인시 처인구 백옥대로 1032  |
|4146110100100060002|경기도 용인시 처인구 백옥대로 1034  |
|4146110100100040002|경기도 용인시 처인구 백옥대로 1034-1|
|4146110100100060010|경기도 용인시 처인구 백옥대로 1038  |
|4146110100100060005|경기도 용인시 처인구 백옥대로 1040  |
+-------------------+------------------------------------+
only showing top 5 rows
SIGUNGU_CODE = 41461


In [5]:
# 날짜 계산 (직전 분기 말일 기준)
today = dt.date.today()
cur_q = (today.month - 1) // 3 + 1
prev_q = cur_q - 1
prev_q_year = today.year
if prev_q == 0:
    prev_q = 4
    prev_q_year -= 1
prev_q_last_month = prev_q * 3

# 데이터 로드 경로
building_path = f"{BUILDING_BASE}/year={prev_q_year}/month={prev_q_last_month:02d}"
toji_path     = f"{TOJI_BASE}/year={prev_q_year}/month={prev_q_last_month:02d}"
rest_owner_path  = f"{REST_OWNER_BASE}/year={prev_q_year}/month={prev_q_last_month:02d}"

In [6]:
# 1) Building
building_df = spark.read.option("mergeSchema", "false").parquet(building_path)
building_gg_df = building_df.filter(F.col("region") == REGION_SHORT)
if SIGUNGU_CODE:
    building_gg_df = building_gg_df.filter(F.col("시군구_코드") == SIGUNGU_CODE)

# 2) Toji
toji_df = spark.read.option("mergeSchema", "false").parquet(toji_path)
toji_gg_df = toji_df.filter(F.col("region") == REGION_SHORT)
if SIGUNGU_CODE:
    toji_gg_df = toji_gg_df.filter(F.col("법정동코드").startswith(SIGUNGU_CODE))

# 3) Owner
owner_df = spark.read.option("mergeSchema", "false").parquet(rest_owner_path)
owner_gg_df = owner_df.filter(F.col("region") == REGION_SHORT)

# Building Pruning (PNU 고유번호 생성 + 필요한 컬럼만 select)

In [7]:
building_gg_df = building_gg_df.withColumn(
    "고유번호",
    F.concat(
        F.col("시군구_코드"),        # 5
        F.col("법정동_코드"),        # 5
        F.when(F.col("대지_구분_코드") == "0", F.lit("1"))  # 대지 -> 1
         .when(F.col("대지_구분_코드") == "1", F.lit("2"))  # 산   -> 2
         .otherwise(F.col("대지_구분_코드")),
        F.lpad(F.col("번"), 4, "0"), # 본번
        F.lpad(F.col("지"), 4, "0")  # 부번
    )
)

building_gg_df = (
    building_gg_df
    .select(
        F.col("관리_건축물대장_PK"),
        F.col("고유번호"),
        F.col("옥외_자주식_면적(㎡)").alias("옥외자주식면적"),
        F.col("대장_구분_코드")
    )
)

building_gg_df.printSchema()
print(f"전체 건축물대장 row 수: {building_gg_df.count()}")
building_gg_df.show(5, truncate=False)

root
 |-- 관리_건축물대장_PK: string (nullable = true)
 |-- 고유번호: string (nullable = true)
 |-- 옥외자주식면적: string (nullable = true)
 |-- 대장_구분_코드: string (nullable = true)

전체 건축물대장 row 수: 51701
+------------------+-------------------+--------------+--------------+
|관리_건축물대장_PK|고유번호           |옥외자주식면적|대장_구분_코드|
+------------------+-------------------+--------------+--------------+
|111615014         |4146125322103440000|0             |1             |
|111616678         |4146125323105250000|0             |1             |
|111619285         |4146126221104650004|0             |2             |
|1116112168        |4146126226100420000|0             |1             |
|111619732         |4146135027104580000|0             |1             |
+------------------+-------------------+--------------+--------------+
only showing top 5 rows


# Toji Pruning (단독+개인 소유, 가장 최근 소유권 변동 정보만 필터링)

In [8]:
all_count = toji_gg_df.count()
print(f"전체 토지 수: {all_count}")

toji_gg_df = (
    toji_gg_df
    # 1) 조건 필터
    .filter(F.col("공유인수") == 0)
    .filter(F.col("소유구분코드") == "01")
    # 2) 필요한 컬럼만 선택
    .select(
        F.col("고유번호").cast("string"),
        F.col("법정동명"),
        F.col("지번"),
        F.col("소유권변동일자"),
        F.col("토지면적"),
        F.col("지목"),
    )

    # 3) 날짜 파싱
    .withColumn(
        "소유권변동일자_dt",
        F.to_date(F.col("소유권변동일자"), "yyyy-MM-dd")
    )

    # 4) 고유번호별 최신 1행
    .withColumn(
        "rn",
        F.row_number().over(
            Window.partitionBy("고유번호")
                  .orderBy(F.col("소유권변동일자_dt").desc_nulls_last())
        )
    )
    .filter(F.col("rn") == 1)
    .drop("rn", "소유권변동일자_dt")

    # 5) 지번 분리
    .withColumn(
        "본번",
        F.split(F.col("지번"), "-").getItem(0)
    )
    .withColumn(
        "부번",
        F.when(
            F.size(F.split(F.col("지번"), "-")) > 1,
            F.split(F.col("지번"), "-").getItem(1)
        ).otherwise(F.lit(None))
    )
)

# 확인
toji_gg_df.printSchema()
print(f"필터링 후 토지 수: {toji_gg_df.count()} ({toji_gg_df.count() / all_count * 100:.2f}%)")
toji_gg_df.show(10, truncate=False)

전체 토지 수: 569885
root
 |-- 고유번호: string (nullable = true)
 |-- 법정동명: string (nullable = true)
 |-- 지번: string (nullable = true)
 |-- 소유권변동일자: string (nullable = true)
 |-- 토지면적: double (nullable = true)
 |-- 지목: string (nullable = true)
 |-- 본번: string (nullable = true)
 |-- 부번: string (nullable = true)



필터링 후 토지 수: 111744 (19.61%)


[Stage 30:=============================>                            (2 + 2) / 4]

+-------------------+-----------------------------+-----+--------------+--------+----+----+----+
|고유번호           |법정동명                     |지번 |소유권변동일자|토지면적|지목|본번|부번|
+-------------------+-----------------------------+-----+--------------+--------+----+----+----+
|4146110100100080001|경기도 용인시 처인구 김량장동|8-1  |2024-09-05    |31.41   |대  |8   |1   |
|4146110100100080004|경기도 용인시 처인구 김량장동|8-4  |2003-03-17    |394.0   |대  |8   |4   |
|4146110100100080011|경기도 용인시 처인구 김량장동|8-11 |2025-07-14    |48.38   |대  |8   |11  |
|4146110100100080013|경기도 용인시 처인구 김량장동|8-13 |2004-08-16    |55.0    |답  |8   |13  |
|4146110100100200017|경기도 용인시 처인구 김량장동|20-17|2016-09-22    |44.0    |대  |20  |17  |
|4146110100100220003|경기도 용인시 처인구 김량장동|22-3 |2002-06-24    |29.0    |도로|22  |3   |
|4146110100100310008|경기도 용인시 처인구 김량장동|31-8 |2016-02-01    |195.0   |대  |31  |8   |
|4146110100100330000|경기도 용인시 처인구 김량장동|33   |1981-08-18    |808.0   |전  |33  |NULL|
|4146110100100390002|경기도 용인시 처인구 김량장동|39-2 |2024-05-21    |49.57   |대  |3

# Restaurant Pruning (필요한 컬럼만 + 중복 제거)

In [9]:
# =========================
# Restaurant pruning
# =========================
# =========================
# Restaurant pruning
# =========================
owner_gg_df_clean = (
    owner_gg_df

    # -----------------------------
    # 1. 괄호 제거
    # -----------------------------
    .withColumn(
        "소재지_tmp",
        F.regexp_replace(F.col("소재지"), r"\s*\([^)]*\)", "")
    )

    # -----------------------------
    # 2. 쉼표 뒤 제거
    # -----------------------------
    .withColumn(
        "소재지_tmp",
        F.regexp_replace(F.col("소재지_tmp"), r",.*$", "")
    )

    # -----------------------------
    # 3. 도로명 앞 행정동 제거
    # (고림동 고림로 123 → 고림로 123)
    # -----------------------------
    .withColumn(
        "소재지_tmp",
        F.regexp_replace(
            F.col("소재지_tmp"),
            r'\s+\S+(동|읍|면|리)\s+(?=\S+(로|길))',
            ' '
        )
    )

    # -----------------------------
    # 4. 공백 정리
    # -----------------------------
    .withColumn(
        "소재지_정제",
        F.trim(F.regexp_replace(F.col("소재지_tmp"), r'\s+', ' '))
    )

    .select(
        F.col("업체명"),
        F.col("대표자"),
        F.col("소재지_정제"),
    )
    .filter(F.col("업체명").isNotNull())
)

# =========================
# Owner 중복 제거
# =========================
all_count = owner_gg_df_clean.count()

owner_gg_df_clean = owner_gg_df_clean.dropDuplicates(["업체명", "소재지_정제"])

print(f"owner (업체명, 소재지) 중복 제거 후: {owner_gg_df_clean.count()} ({owner_gg_df_clean.count() / all_count * 100:.2f}%)")

owner_gg_df_clean.show(10, truncate=False)

owner (업체명, 소재지) 중복 제거 후: 5453 (98.71%)
+----------------------------+------+-------------------------------------------+
|업체명                      |대표자|소재지_정제                                |
+----------------------------+------+-------------------------------------------+
|(B.H.C)비에이치씨 용인송전점|김*숙 |경기도 용인시 처인구 경기동로687번길 6     |
|(BHC)비에이치씨둔전점       |김*정 |경기도 용인시 처인구 포곡로108번길 5-13    |
|(BHC)비에이치씨역북점       |김*하 |경기도 용인시 처인구 중부대로1281번길 10-29|
|(顥)호돼지네                |용*귀 |경기도 용인시 처인구 경안천로 232          |
|(사)천주교인보회요한의집    |한*란 |경기도 용인시 처인구 백옥대로1832번길 58   |
|(사)천주교인보회인보마을    |곽*리 |경기도 용인시 처인구 백옥대로1832번길 42   |
|(의)영문의료재단 다보스병원 |양*범 |경기도 용인시 처인구 백옥대로1082번길 18   |
|(주)SCD                     |오*호 |경기도 용인시 처인구 형제로17번길 21       |
|(주)갈비명가소들녘          |전*민 |경기도 용인시 처인구 양지로 242            |
|(주)기가텍                  |이*대 |경기도 용인시 처인구 대지로 409-15         |
+----------------------------+------+-------------------------------------------+
only showing top 10 rows


# Restaurant + Address Left Join (도로명주소 -> 법정동코드 컬럼 추가)

In [10]:
o = owner_gg_df_clean.alias("o")
a = address_filtered_df.alias("a")

restaurant_address_df = (
    o.join(
        a,
        F.col("o.소재지_정제") == F.col("a.도로명주소"),
        how="left"
    )
    .filter(F.col("도로명주소").isNotNull())
    .drop("도로명주소")
)

restaurant_address_df.printSchema()
print(f"owner 도로명주소 매칭 후: {restaurant_address_df.count()} ({restaurant_address_df.count() / owner_gg_df_clean.count() * 100:.2f}%)")
restaurant_address_df.show(10, truncate=False)

root
 |-- 업체명: string (nullable = true)
 |-- 대표자: string (nullable = true)
 |-- 소재지_정제: string (nullable = true)
 |-- PNU코드: string (nullable = true)

owner 도로명주소 매칭 후: 5325 (97.65%)
+--------------------------+-------+----------------------------------+-------------------+
|업체명                    |대표자 |소재지_정제                       |PNU코드            |
+--------------------------+-------+----------------------------------+-------------------+
|토종흑염소전문            |전*규  |경기도 용인시 처인구 백옥대로 1048|4146110100100090005|
|은희네 행복한 밥집        |이*나라|경기도 용인시 처인구 백옥대로 1048|4146110100100090005|
|봉산짬뽕                  |김*두  |경기도 용인시 처인구 백옥대로 1048|4146110100100090005|
|복가                      |임*진  |경기도 용인시 처인구 백옥대로 1058|4146110100100120015|
|GS25용인태성점            |이*용  |경기도 용인시 처인구 백옥대로 1059|4146110100100310011|
|자마르(ZAMAR)             |최*영  |경기도 용인시 처인구 백옥대로 1066|4146110100100130000|
|롯데쇼핑(주)롯데슈퍼용인점|김*재  |경기도 용인시 처인구 백옥대로 1066|4146110100100130000|
|동해횟집                  |황*희  |경기도 용인시 처인구 백옥대로 1072|41461

# 토지 필터링
1. 토지에 건물이 없거나
2. 건물이 1개 있고, 일반 건물이며, 음식점이 있는 토지만 필터링

In [11]:
t = toji_gg_df.alias("t")
b = building_gg_df.alias("b")

toji_building_gg_df = (
    t.join(b, F.col("t.고유번호") == F.col("b.고유번호"), "left")
     .drop(F.col("b.고유번호"))
)

toji_building_gg_df.printSchema()
print(toji_building_gg_df.count())
toji_building_gg_df.show(5, truncate=False)

root
 |-- 고유번호: string (nullable = true)
 |-- 법정동명: string (nullable = true)
 |-- 지번: string (nullable = true)
 |-- 소유권변동일자: string (nullable = true)
 |-- 토지면적: double (nullable = true)
 |-- 지목: string (nullable = true)
 |-- 본번: string (nullable = true)
 |-- 부번: string (nullable = true)
 |-- 관리_건축물대장_PK: string (nullable = true)
 |-- 옥외자주식면적: string (nullable = true)
 |-- 대장_구분_코드: string (nullable = true)

118721


[Stage 84:==============================================>           (4 + 1) / 5]

+-------------------+-----------------------------+----+--------------+--------+----+----+----+------------------+--------------+--------------+
|고유번호           |법정동명                     |지번|소유권변동일자|토지면적|지목|본번|부번|관리_건축물대장_PK|옥외자주식면적|대장_구분_코드|
+-------------------+-----------------------------+----+--------------+--------+----+----+----+------------------+--------------+--------------+
|4146110100100080001|경기도 용인시 처인구 김량장동|8-1 |2024-09-05    |31.41   |대  |8   |1   |11161100478093    |0             |2             |
|4146110100100080001|경기도 용인시 처인구 김량장동|8-1 |2024-09-05    |31.41   |대  |8   |1   |11161100478078    |0             |2             |
|4146110100100080004|경기도 용인시 처인구 김량장동|8-4 |2003-03-17    |394.0   |대  |8   |4   |1116116027        |0             |1             |
|4146110100100080004|경기도 용인시 처인구 김량장동|8-4 |2003-03-17    |394.0   |대  |8   |4   |1116116026        |0             |1             |
|4146110100100080011|경기도 용인시 처인구 김량장동|8-11|2025-07-14    |48.38   |대  |8   |11  |1116136

### 건물이 1개 이하인 토지 필터링

In [12]:
pk_cnt_df = (
    toji_building_gg_df
    .groupBy("고유번호")
    .agg(F.count("관리_건축물대장_PK").alias("pk_cnt"))
)

toji_binary_building_gg_df = (
    toji_building_gg_df
    .join(
        pk_cnt_df.filter(F.col("pk_cnt") <= 1),
        on="고유번호",
        how="inner"
    )
    .drop("pk_cnt")
)

print(toji_binary_building_gg_df.count())
toji_binary_building_gg_df.show(5, truncate=False)

107862


+-------------------+-----------------------------+-----+--------------+--------+----+----+----+------------------+--------------+--------------+
|고유번호           |법정동명                     |지번 |소유권변동일자|토지면적|지목|본번|부번|관리_건축물대장_PK|옥외자주식면적|대장_구분_코드|
+-------------------+-----------------------------+-----+--------------+--------+----+----+----+------------------+--------------+--------------+
|4146110100100080011|경기도 용인시 처인구 김량장동|8-11 |2025-07-14    |48.38   |대  |8   |11  |1116136285        |92            |2             |
|4146110100100080013|경기도 용인시 처인구 김량장동|8-13 |2004-08-16    |55.0    |답  |8   |13  |NULL              |NULL          |NULL          |
|4146110100100200017|경기도 용인시 처인구 김량장동|20-17|2016-09-22    |44.0    |대  |20  |17  |NULL              |NULL          |NULL          |
|4146110100100220003|경기도 용인시 처인구 김량장동|22-3 |2002-06-24    |29.0    |도로|22  |3   |NULL              |NULL          |NULL          |
|4146110100100310008|경기도 용인시 처인구 김량장동|31-8 |2016-02-01    |195.0   |대  |31  |8   |

In [13]:
toji_binary_building_gg_df.withColumn(
    "건물개수",
    F.when(F.col("관리_건축물대장_PK").isNull(), F.lit(0))
     .otherwise(F.lit(1))
).groupBy("건물개수").count().orderBy("건물개수").show()

+--------+-----+
|건물개수|count|
+--------+-----+
|       0|90264|
|       1|17598|
+--------+-----+



### 건물이 1개 있고, 일반 건물이며, 음식점이 포함된 토지 필터링

In [14]:
# 건물이 1개 있는 토지만 필터링
toji_with_1_building_gg_df = (
    toji_binary_building_gg_df
    .filter(F.col("관리_건축물대장_PK").isNotNull())
)

print("건물 1개인 토지 수:", toji_with_1_building_gg_df.count())

건물 1개인 토지 수: 17598


In [15]:
# 건물 있는데 음식점 아닌거 거르려고 만드는 join table

t = toji_with_1_building_gg_df.alias("t")
r = restaurant_address_df.alias("r")

toji_building_restaurant_gg_df = (
    t.join(
        r,
        F.col("t.고유번호") == F.col("r.PNU코드"),
        how="left"
    )
    .drop(F.col("r.PNU코드"))
    .filter(F.col("업체명").isNotNull())
)

print("건물이 1개이고, 음식점이 있는 토지:", toji_building_restaurant_gg_df.count())
toji_building_restaurant_gg_df.show(5)

건물이 1개이고, 음식점이 있는 토지: 2223


+-------------------+-----------------------------+-------+--------------+--------+----+----+----+------------------+--------------+--------------+------------+---------+--------------------------------+
|           고유번호|                     법정동명|   지번|소유권변동일자|토지면적|지목|본번|부번|관리_건축물대장_PK|옥외자주식면적|대장_구분_코드|      업체명|   대표자|                     소재지_정제|
+-------------------+-----------------------------+-------+--------------+--------+----+----+----+------------------+--------------+--------------+------------+---------+--------------------------------+
|4146110100101330060|경기도 용인시 처인구 김량장동| 133-60|    2008-04-24|   106.0|  대| 133|  60|        1116136484|             0|             1|    고기창고|    김*하|경기도 용인시 처인구 금령로85...|
|4146110100101330060|경기도 용인시 처인구 김량장동| 133-60|    2008-04-24|   106.0|  대| 133|  60|        1116136484|             0|             1|      엠와이|    장*애|경기도 용인시 처인구 금령로85...|
|4146110100101330123|경기도 용인시 처인구 김량장동|133-123|    2015-11-10|   152.0|  대| 133| 123|        111612962

### 건물 0개 + 건물 1개 (일반건물, 음식점있음) 토지 concat

In [16]:
# 아무 건물도 없는 필지
toji_with_0_building_gg_df = (
    toji_binary_building_gg_df
    .filter(F.col("관리_건축물대장_PK").isNull())
)

# 위에서 만든 join table과 concat을 위해 column 추가
toji_with_0_building_gg_df = (
    toji_with_0_building_gg_df
    .withColumn("업체명", F.lit(None).cast("string"))
    .withColumn("대표자", F.lit(None).cast("string"))
    .withColumn("소재지_정제", F.lit(None).cast("string"))
)

final_toji_df = (
    toji_building_restaurant_gg_df
    .unionByName(toji_with_0_building_gg_df)
)

In [17]:
# 전체 row 수 = (건물1+음식점) + (건물0)
print("final:", final_toji_df.count())
print("건물1+음식점:", toji_building_restaurant_gg_df.count())
print("건물0:", toji_with_0_building_gg_df.count())

# 건물 개수 분포
final_toji_df.withColumn(
    "건물개수",
    F.when(F.col("관리_건축물대장_PK").isNull(), 0).otherwise(1)
).groupBy("건물개수").count().orderBy("건물개수").show()

final_toji_df.show(5)

final: 92487


건물1+음식점: 2223
건물0: 90264


+--------+-----+
|건물개수|count|
+--------+-----+
|       0|90264|
|       1| 2223|
+--------+-----+



26/02/16 10:14:54 ERROR TaskSchedulerImpl: Lost executor 0 on 172.22.0.4: Command exited with code 137
26/02/16 10:14:54 WARN TaskSetManager: Lost task 0.0 in stage 239.0 (TID 349) (172.22.0.4 executor 0): ExecutorLostFailure (executor 0 exited caused by one of the running tasks) Reason: Command exited with code 137
26/02/16 10:14:54 WARN TaskSetManager: Lost task 2.0 in stage 239.0 (TID 351) (172.22.0.4 executor 0): ExecutorLostFailure (executor 0 exited caused by one of the running tasks) Reason: Command exited with code 137
26/02/16 10:14:56 WARN TaskSetManager: Lost task 0.0 in stage 246.0 (TID 366) (172.22.0.3 executor 1): FetchFailed(null, shuffleId=75, mapIndex=-1, mapId=-1, reduceId=0, message=
org.apache.spark.shuffle.MetadataFetchFailedException: Missing an output location for shuffle 75 partition 0
	at org.apache.spark.MapOutputTracker$.validateStatus(MapOutputTracker.scala:1781)
	at org.apache.spark.MapOutputTracker$.$anonfun$convertMapStatuses$11(MapOutputTracker.scala:172

+-------------------+-----------------------------+-----+--------------+--------+----+----+----+------------------+--------------+--------------+-------------------------------+--------------------+---------------------------------+
|           고유번호|                     법정동명| 지번|소유권변동일자|토지면적|지목|본번|부번|관리_건축물대장_PK|옥외자주식면적|대장_구분_코드|                         업체명|              대표자|                      소재지_정제|
+-------------------+-----------------------------+-----+--------------+--------+----+----+----+------------------+--------------+--------------+-------------------------------+--------------------+---------------------------------+
|4146110100100310011|경기도 용인시 처인구 김량장동|31-11|    2015-06-16|   139.0|  대|  31|  11|        1116119256|          11.5|             1|                 GS25용인태성점|               이*용|경기도 용인시 처인구 백옥대로 ...|
|4146110100100410005|경기도 용인시 처인구 김량장동| 41-5|    1996-05-15|   340.0|  대|  41|   5|        1116121579|             0|             1|                       차곡차곡| 

# 같은 본번을 가진 토지 그룹핑 -> 음식점이 있는 그룹만 필터링

In [25]:
group_has_restaurant_df = (
    final_toji_df
    .groupBy("법정동명", "본번")
    .agg(
        F.max(
            F.when(F.col("업체명").isNotNull(), 1).otherwise(0)
        ).alias("has_restaurant")
    )
    .filter(F.col("has_restaurant") == 1)
    .select("법정동명", "본번")
)

filtered_final_toji_df = (
    final_toji_df
    .join(group_has_restaurant_df, on=["법정동명", "본번"], how="inner")
)

print("필터 후 row 수:", filtered_final_toji_df.count())
filtered_final_toji_df.show(5, truncate=False)

필터 후 row 수: 5740


[Stage 624:>                                                        (0 + 4) / 4]

+-----------------------------+----+-------------------+-----+--------------+--------+----+----+------------------+--------------+--------------+--------------------------+------+--------------------------------------+
|법정동명                     |본번|고유번호           |지번 |소유권변동일자|토지면적|지목|부번|관리_건축물대장_PK|옥외자주식면적|대장_구분_코드|업체명                    |대표자|소재지_정제                           |
+-----------------------------+----+-------------------+-----+--------------+--------+----+----+------------------+--------------+--------------+--------------------------+------+--------------------------------------+
|경기도 용인시 처인구 김량장동|333 |4146110100103330001|333-1|1987-06-24    |226.1   |대  |1   |1116126076        |0             |1             |보석포차                  |유*숙 |경기도 용인시 처인구 금령로 24        |
|경기도 용인시 처인구 해곡동  |175 |4146111000101750003|175-3|2015-09-09    |600.0   |대  |3   |1116111330        |0             |1             |엉끌드파리(uncle de paris)|김*희 |경기도 용인시 처인구 해곡로 30        |
|경기도 용인시 처인구 역북동  |754 |414

# 같은 지주 소유로 추정되는 토지 그룹핑 -> 토지대장 발급 리스트
* 같은 본번으로 묶이는 토지 내에서 소유권변동일자가 같으면 같은 사람이 매입한 것으로 추정

In [29]:
toji_group_df = (
    filtered_final_toji_df
    .groupBy("법정동명", "본번", "소유권변동일자")
    .agg(
        F.min("부번").alias("부번리스트")
    )
    .distinct()
)

print(toji_group_df.count())
toji_group_df.show(10, truncate=False)

3343


26/02/16 10:44:16 ERROR TaskSchedulerImpl: Lost executor 6 on 172.22.0.4: Command exited with code 137
26/02/16 10:44:16 WARN TaskSetManager: Lost task 0.0 in stage 902.0 (TID 1306) (172.22.0.3 executor 7): FetchFailed(null, shuffleId=283, mapIndex=-1, mapId=-1, reduceId=0, message=
org.apache.spark.shuffle.MetadataFetchFailedException: Missing an output location for shuffle 283 partition 0
	at org.apache.spark.MapOutputTracker$.validateStatus(MapOutputTracker.scala:1781)
	at org.apache.spark.MapOutputTracker$.$anonfun$convertMapStatuses$11(MapOutputTracker.scala:1726)
	at org.apache.spark.MapOutputTracker$.$anonfun$convertMapStatuses$11$adapted(MapOutputTracker.scala:1725)
	at scala.collection.IterableOnceOps.foreach(IterableOnce.scala:630)
	at scala.collection.IterableOnceOps.foreach$(IterableOnce.scala:628)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1313)
	at org.apache.spark.MapOutputTracker$.convertMapStatuses(MapOutputTracker.scala:1725)
	at org.apache.spark.Map

+---------------------------+----+--------------+----------+
|법정동명                   |본번|소유권변동일자|부번리스트|
+---------------------------+----+--------------+----------+
|경기도 용인시 처인구 고림동|193 |2023-12-26    |4         |
|경기도 용인시 처인구 고림동|208 |2006-11-17    |14        |
|경기도 용인시 처인구 고림동|208 |2025-02-28    |11        |
|경기도 용인시 처인구 고림동|264 |2025-05-08    |18        |
|경기도 용인시 처인구 고림동|264 |2025-07-30    |3         |
|경기도 용인시 처인구 고림동|264 |2025-10-16    |1         |
|경기도 용인시 처인구 고림동|266 |2012-12-27    |1         |
|경기도 용인시 처인구 고림동|341 |2019-10-31    |6         |
|경기도 용인시 처인구 고림동|341 |2022-05-04    |2         |
|경기도 용인시 처인구 고림동|342 |1998-04-01    |1         |
+---------------------------+----+--------------+----------+
only showing top 10 rows


In [34]:
# =========================
# 저장 경로 생성
# =========================
base_output_path = os.path.join(
    OUTPUT_BASE,
    f"{REGION}_{SIGUNGU.replace(' ', '_')}"
)

filtered_path = base_output_path + "_filtered_final_toji"
group_path = base_output_path + "_toji_group"

print("filtered 저장 경로:", filtered_path)
print("group 저장 경로:", group_path)

# =========================
# parquet 저장
# =========================
filtered_final_toji_df.write \
    .mode("overwrite") \
    .parquet(filtered_path)

toji_group_df.write \
    .mode("overwrite") \
    .parquet(group_path)

print("✅ 저장 완료")


filtered 저장 경로: /opt/spark/data/output/silver_stage_1/경기도_용인시_처인구_filtered_final_toji
group 저장 경로: /opt/spark/data/output/silver_stage_1/경기도_용인시_처인구_toji_group


26/02/16 10:58:04 ERROR TaskSchedulerImpl: Lost executor 8 on 172.22.0.4: Command exited with code 137
26/02/16 10:58:04 WARN TaskSetManager: Lost task 0.0 in stage 1076.0 (TID 1605) (172.22.0.4 executor 8): ExecutorLostFailure (executor 8 exited caused by one of the running tasks) Reason: Command exited with code 137
26/02/16 10:58:04 WARN TaskSetManager: Lost task 1.0 in stage 1076.0 (TID 1606) (172.22.0.4 executor 8): ExecutorLostFailure (executor 8 exited caused by one of the running tasks) Reason: Command exited with code 137
26/02/16 10:58:05 WARN TaskSetManager: Lost task 3.0 in stage 1072.0 (TID 1608) (172.22.0.3 executor 9): FetchFailed(BlockManagerId(8, 172.22.0.4, 44389, None), shuffleId=332, mapIndex=2, mapId=1560, reduceId=200, message=
org.apache.spark.shuffle.FetchFailedException
	at org.apache.spark.errors.SparkCoreErrors$.fetchFailedError(SparkCoreErrors.scala:426)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.throwFetchFailedException(ShuffleBlockFetcherIte

✅ 저장 완료


In [35]:
spark.stop()